# Import data from Conflicted Glossary

In [1]:
import pandas as pd

# Load the CSV files into DataFrames
codes_df = pd.read_csv('../_temp/glossary.csv')  # type: ignore
transcripts_df = pd.read_csv('../_temp/transcripts.csv')  # type: ignore

# Rename columns according to the provided mapping
codes_df = codes_df.rename(columns={
    'Glossary ID': 'code_id',
    'Term': 'term',
    'Description': 'description',
    'Type': 'type',
    'Read more': 'reference',
    'Lat/Long': 'coordinates'
})

transcripts_df = transcripts_df.rename(columns={
    'Text ID': 'transcript_id',
    'Text': 'transcript_text',
    'Episode ID': 'episode_id',
    'Episode': 'episode_title',
    'Glossary IDs': 'code_ids'
})
# drop the columns that are not transcript_id, transcript_text, episode_id, episode_title, code_ids
transcripts_df = transcripts_df[['transcript_id', 'transcript_text', 'episode_id', 'episode_title', 'code_ids']]

transcripts_df['code_ids'] = transcripts_df['code_ids'].fillna('') # type: ignore

# create a dataframe with all unique episode_id and episode_title
episode_df = transcripts_df[['episode_id', 'episode_title']].drop_duplicates()

# create a dataframe with all unique glossary type
code_type_df = codes_df[['type']].drop_duplicates()
# rename the column to type_name
code_type_df = code_type_df.rename(columns={'type': 'type_name'})
# create an incrementing type_id column
code_type_df['type_id'] = range(1, len(code_type_df) + 1)

# Create a column type_id in the codes_df dataframe and map the type_name to the type_id
# Merge the code_type_df with the codes_df on type = type_name
codes_df = codes_df.merge(code_type_df, left_on='type', right_on='type_name', how='left') # type: ignore
codes_df = codes_df.rename(columns={'type_name': 'type_id'})
codes_df = codes_df[['code_id', 'term', 'description', 'type_id', 'reference', 'coordinates']]

In [2]:
# Create a new DataFrame annotations_df with columns transcript_id, code_id
annotations_df = pd.DataFrame(columns=['transcript_id', 'code_id'])

# Iterate over each row in transcripts_df and add rows to annotations_df
rows_to_add = []
for index, row in transcripts_df.iterrows():
    transcript_id = row['transcript_id']
    code_ids = row['code_ids'].split(';')
    for code_id in code_ids if code_ids != [''] else []:
        code_id = int(code_id)
        rows_to_add.append({'transcript_id': transcript_id, 'code_id': code_id})

transcripts_df.drop(columns=['code_ids'], inplace=True) # type: ignore

# Convert list of rows to DataFrame and concatenate
annotations_df = pd.concat([annotations_df, pd.DataFrame(rows_to_add)], ignore_index=True)

# Convert code_id to int
annotations_df['code_id'] = annotations_df['code_id'].astype(int)

# Add a column annotation_id to annotations_df
annotations_df['annotation_id'] = annotations_df.index

# Define Database

In [3]:
from kanot.db.schema import create_database, drop_database, CodeType, Code, Episode, Transcript, Annotation
from sqlalchemy import create_engine, Engine
from sqlalchemy.orm import sessionmaker

# Create a connection to the SQLite database
engine: Engine = create_engine('sqlite:///../local_database.db', echo=True)
MySession = sessionmaker(bind=engine)
session = MySession()

drop = True
if drop:
    # Drop all the tables in the database
    drop_database(engine)

# Create the tables in the database
create_database(engine)

2024-07-15 12:36:22,061 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-15 12:36:22,061 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("code_types")
2024-07-15 12:36:22,061 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 12:36:22,062 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("codes")
2024-07-15 12:36:22,062 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 12:36:22,063 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("episodes")
2024-07-15 12:36:22,063 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 12:36:22,063 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("transcripts")
2024-07-15 12:36:22,063 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 12:36:22,063 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("annotations")
2024-07-15 12:36:22,064 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-07-15 12:36:22,064 INFO sqlalchemy.engine.Engine 
DROP TABLE annotations
2024-07-15 12:36:22,064 INFO sqlalchemy.engine.Engine [no key 0.00

In [4]:
from sqlalchemy import func
from sqlalchemy.orm import Session
from sqlalchemy.ext.declarative import DeclarativeMeta
from sqlalchemy.exc import IntegrityError
from typing import Any, Hashable

# Define a function to insert data from a DataFrame
def insert_data(df: pd.DataFrame, model: DeclarativeMeta, session: Session) -> None:
    data: list[dict[Hashable, Any]] = df.to_dict(orient='records') # type: ignore
    
    try:
        session.bulk_insert_mappings(model, data) # type: ignore
        session.commit()
    except IntegrityError as e:
        print(f"Unique constraint violation: {e}")
        session.rollback()

# Insert data into the code_types table
insert_data(code_type_df, CodeType, session)

# Insert data into the codes table
insert_data(codes_df, Code, session)

# Insert data into the episodes table
insert_data(episode_df, Episode, session)

# Insert data into the transcripts table
insert_data(transcripts_df, Transcript, session)

# Insert data into the annotations table
insert_data(annotations_df, Annotation, session)

# Close the session
session.close()

2024-07-15 12:36:22,090 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-15 12:36:22,091 INFO sqlalchemy.engine.Engine INSERT INTO code_types (type_id, type_name) VALUES (?, ?)
2024-07-15 12:36:22,091 INFO sqlalchemy.engine.Engine [generated in 0.00027s] [(1, 'Person'), (2, 'Events and Periods'), (3, 'Group'), (4, 'Concept: Cultural and Societal'), (5, 'Geography'), (6, 'Concept: Political and Social'), (7, 'Concept: Economics and Infrastructure'), (8, 'Concept: Terrorism and Security')  ... displaying 10 of 16 total bound parameter sets ...  (15, 'Education and Institutions'), (16, 'Concept: Legal')]
2024-07-15 12:36:22,092 INFO sqlalchemy.engine.Engine COMMIT
2024-07-15 12:36:22,094 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-15 12:36:22,096 INFO sqlalchemy.engine.Engine INSERT INTO codes (code_id, term, description, type_id, reference, coordinates) VALUES (?, ?, ?, ?, ?, ?)
2024-07-15 12:36:22,096 INFO sqlalchemy.engine.Engine [generated in 0.00095s] [(875, 'Zelimkh

/var/folders/qf/j616b9d53yn77zcpky42s5r40000gn/T/ipykernel_96845/2435014864.py:9: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  data: list[dict[Hashable, Any]] = df.to_dict(orient='records') # type: ignore


In [5]:
# Step 1: Select a random code item
random_code = session.query(Code).order_by(func.random()).first()

if random_code:
    code_id = random_code.code_id
    print("Random Code ID:")
    print(code_id)
    print("Random Code Item:")
    print(random_code)

    # Step 2: Find all annotations associated with the selected code
    annotations = session.query(Annotation).filter(Annotation.code_id == code_id).all()

    if annotations:
        print('----\nRESULTS\n-----')
        for annotation in annotations:
            print("\nAssociated Annotation:")
            print(annotation)

            # Step 3: Retrieve all transcripts associated with those annotations
            transcript_id = annotation.transcript_id
            transcript = session.query(Transcript).filter(Transcript.transcript_id == transcript_id).first()

            if transcript:
                print("\nAssociated Transcript:")
                print(transcript)
            else:
                print("No associated transcripts found.")
            print("\n")
            print("--------------------------------------------------")
            print("\n")
    else:
        print("No associated annotations found.")
else:
    print("No codes found in the database.")


2024-07-15 12:36:22,210 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-07-15 12:36:22,212 INFO sqlalchemy.engine.Engine SELECT codes.code_id AS codes_code_id, codes.term AS codes_term, codes.description AS codes_description, codes.type_id AS codes_type_id, codes.reference AS codes_reference, codes.coordinates AS codes_coordinates 
FROM codes ORDER BY random()
 LIMIT ? OFFSET ?
2024-07-15 12:36:22,212 INFO sqlalchemy.engine.Engine [generated in 0.00046s] (1, 0)
Random Code ID:
114
Random Code Item:
Code(code_id=114, term=Jemaah Islamiyah, description=An Islamist militant group based in Southeast Asia, responsible for several terrorist attacks and known for its radical views., type_id=3, reference=https://en.wikipedia.org/wiki/Jemaah_Islamiyah, coordinates=None)
2024-07-15 12:36:22,214 INFO sqlalchemy.engine.Engine SELECT annotations.annotation_id AS annotations_annotation_id, annotations.transcript_id AS annotations_transcript_id, annotations.code_id AS annotations_code_id 
FROM an